<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/ML2023/blob/main/PokemonClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar as bibliotecas

In [ ]:
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

#Preparar o Dataset


Ler o data set

In [ ]:
pokemonDF = pd.read_csv("/content/pokemon.csv")

Préprocessar

## Remover as entradas que não deveriam pertencer ao dataset.

Aqui removemos as linhas que o enunciado solicita que nós não utlizemos. O que é uma pena, o conhecimento a priório de pokémon diz que além de fraqueza contra elétrico, pokémons aquaticos tem fraqueza contra grama. Essa seria um ótimo parametro para selecionar pokémons como Swampert, Wishcash e Seismictoad, cujos tipos secundários (terra) conferem imunidade ao tipo elétrico.

Além disso, discartar o tipo secundário, resulta em um dataset imperfeito, já que a ordem do tipo do pokémon é inter operavel, vide: https://www.reddit.com/r/pokemon/comments/b2zvom/primary_vs_secondary_types/

 Dois pokémons pode ter o mesmo tipo em ordem reversa, como sableye (Dark/Ghost) e Spiritomb (Ghost/Dark). Além disso, pokémons podem ter o tipo pertinente como o segundo tipo, que são desconsiderados.

Além disso temos um exemplo que pertence a ambas as classes, Bibarel, que tem ambos os tipos, que por nossa classificacao, pertenceria ao tipo normal, mas que igualmente poderia ser classificado como tipo agua. Por ele ser um outlier nesse aspecto, ele foi removido.





In [ ]:
# remover linhas "ilegais"
pokemonDF.drop(["abilities", "against_bug", "against_dark",	"against_dragon",	"against_fairy",	"against_fight",	"against_fire",	"against_flying",	"against_ghost", "against_grass","against_ground","against_ice","against_normal","against_poison","against_psychic","against_rock", "against_steel", "against_water", "type2"], axis =1)

,against_electric,attack,base_egg_steps,base_happiness,base_total,capture_rate,classfication,defense,experience_growth,height_m,...,name,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,weight_kg,generation,is_legendary
0,0.5,49,5120,70,318,45,Seed Pokémon,49,1059860,0.7,...,Bulbasaur,88.1,1,65,65,45,grass,6.9,1,0
1,0.5,62,5120,70,405,45,Seed Pokémon,63,1059860,1.0,...,Ivysaur,88.1,2,80,80,60,grass,13.0,1,0
2,0.5,100,5120,70,625,45,Seed Pokémon,123,1059860,2.0,...,Venusaur,88.1,3,122,120,80,grass,100.0,1,0
3,1.0,52,5120,70,309,45,Lizard Pokémon,43,1059860,0.6,...,Charmander,88.1,4,60,50,65,fire,8.5,1,0
4,1.0,64,5120,70,405,45,Flame Pokémon,58,1059860,1.1,...,Charmeleon,88.1,5,80,65,80,fire,19.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,2.0,101,30720,0,570,25,Launch Pokémon,103,1250000,9.2,...,Celesteela,NaN,797,107,101,61,steel,999.9,7,1
797,0.5,181,30720,0,570,255,Drawn Sword Pokémon,131,1250000,0.3,...,Kartana,NaN,798,59,31,109,grass,0.1,7,1
798,0.5,101,30720,0,570,15,Junkivore Pokémon,53,1250000,5.5,...,Guzzlord,NaN,799,97,53,43,dark,888.0,7,1
799,1.0,107,30720,0,600,3,Prism Pokémon,101,1250000,2.4,...,Necrozma,NaN,800,127,89,79,psychic,230.0,7,1


A

## Analise dos dados

#Aprendizado

Vizualizacão